# Ease of social distancing index v1.0
## Notebook 2: calculation of index

Code written by Heather Chamberlain

This notebook contains the code needed to calculate the ease of social distancing index value for each spatial unit. The spatial units can be created using Notebook 1. 

Once this Notebook has been run to completion, the output should consist of the ease of social distancing index values calculated for all spatial units, along with contributing built and population density scores. 

In [ ]:
import fnmatch
import os
import arcpy
import glob

from arcpy.sa import *
arcpy.CheckOutExtension("Spatial")

from pathlib import Path

### 1. Setup the necessary filepaths and directory structure

Specify country of interest using 3 letter ISO code, and specify path for home directory

In [ ]:
iso = "KEN"
folder_iso = "XXX"
#e.g. iso = "SSD"

home_folder = Path("SUBSTITUTE/FOLDER/PATH/HERE")

print('ISO code specified: ' + str(iso))

In [ ]:
#dataset version
vers = 'v1_0'

#working gdb name
gdb_name = str(iso) + "_working_v1_0_alpha.gdb"

In [ ]:
c_folder = home_folder / folder_iso #path for country folder

ssa_folder = home_folder / "ssa51" #path for folder with data for all 51 countries in Sub-Saharan Africa

data_in_folder = c_folder / "DataIn" #path for input data directory for the country of interest
w_folder = c_folder / "Working" #working directory for the country of interest
output_folder = c_folder / "Output" #output directory for the country of interest

uext_iso_folder = data_in_folder / "AOIs"  #path for sub-directory for urban extent (AOI) file 
adm0_iso_folder = data_in_folder / "Adm0"  #path for sub-directory for national boundary file for the country of interest
utm_iso_folder = data_in_folder / "UTMzone_polys"  #path for sub-directory for UTM zone file

bf_root = Path("SUBSTITUTE/FOLDER/PATH/HERE")
iso_bf_folder = data_in_folder / "BuildingFootprints" #path for sub-directory for additional building footprint files for the country of interest

In [ ]:
gdb_path = w_folder / gdb_name
if os.path.isdir(str(gdb_path)):
    print("w_gdb already exists")
else:
    print("creating w_gdb")
    arcpy.CreateFileGDB_management(str(w_folder), gdb_name) #create gdb
w_gdb = w_folder / gdb_name

### 2. Identify the input files needed

Load the data needed to calculate the population density and built score values:
- Spatial units (created from Notebook 1)
- Building footprints (Maxar/Ecopia building footprints were used in calculating the ease of social distancing index v1.0, but other similar datasets could be used in their place where available e.g. Google building footprints, Microsoft building footprints)
- WorldPop gridded population datasets
- Grid cell surface area dataset (providing area of each grid cell in the WP gridded pop datasets)

Other files required in processing data:
- UTM zones
- Urban extents (AOIs)

#### Load the data needed to calculate the population density and built score values

First identify the UTM zones in which the spatial units are located - 1 file per UTM zone

In [ ]:
listing = glob.glob(str(utm_iso_folder) + "/**/UTMzone_*.shp", recursive = True)

utm_zones_list = []

for filename in listing:
    u = (Path(filename).stem).split("_")[1]
    utm_zones_list.append(u)   
    
s_count = len(utm_zones_list)

print("for " + str(iso) + " the spatial units are split across " + str(s_count) + " UTM zones: " + str(utm_zones_list))

Spatial units

In [ ]:
spatial_units = []

for zone in utm_zones_list:
    area_polys_elimf = str(w_gdb / ("area_polys_elimf_UTM" + str(zone)))
    spatial_units.append(area_polys_elimf)
    
print(spatial_units)

Building footprints

In [ ]:
#building footprints
bf_path = bf_root / iso #for one country, different datasets for different UTM zones.

blist = []

bf_gdb = glob.glob(str(bf_path) + "/**/*.gdb", recursive = True)
folder = Path(bf_gdb[0]).parent

for file in os.listdir(folder):
    if fnmatch.fnmatch(file, '*.shp'):
        blist.append(str(folder / file))
        #print(file) 
                     
print(blist)

Population rasters

In [ ]:
#pop_raster 
pop_path = home_folder / iso / "DataIn" / "PopRaster" #pop raster for one country.
    
popbu_list = ['ZMB', 'NGA', 'SLE', 'MOZ', 'SSD', 'GHA', 'COD', 'BFA'] #list of iso codes for countries with bottom-up pop datasets/alternatives to the global constrained datasets

#different file names for top-down and bottom-up datasets. Use if statement to get appropriate name format
if iso.upper() in popbu_list:
    for file in os.listdir(pop_path):
        if fnmatch.fnmatch(file, '*gridded.tif'):
            pop_file_name = file
            pop_raster = str(pop_path / pop_file_name)
            print('bottom up pop dataset: {}'.format(pop_raster))
        elif fnmatch.fnmatch(file, '*gridded_population.tif'):
            pop_file_name = file
            pop_raster = str(pop_path / pop_file_name)
            print('bottom up pop dataset: {}'.format(pop_raster))
else:
    pop_file_name = str(iso.lower()) + "_ppp_2020_constrained.tif"
    pop_raster = str(pop_path / pop_file_name)
    print('top down constrained pop dataset: {}'.format(pop_raster))

Grid cell surface area raster (used in converting pop count to density)

In [ ]:
#grid cell surface area dataset
px_file_name = str(iso.lower()) + "_px_area_100m.tif"
px_area_raster = str(pop_path / px_file_name)
print('px area raster: {}'.format(px_area_raster))

#### Other files required in processing data

In [ ]:
#urban extent polygons (clipped to adm0 boundary)
uext_poly = str(uext_iso_folder / ("uext_polys_" + str(iso) + "_adm0.shp"))

In [ ]:
#utm zone boundaries
utm_folder = home_folder / "UTM"
utm_zones = str(utm_folder / "UTM_zones_all.shp")

### 3. Prepare the input files needed for calculating the population density and built score values

Clip building footprints to urban extents

In [ ]:
#clip bfs to uext
clip_area = str(uext_poly)
        
for file in blist:    
    data_in = file
    #print(data_in)
    data_out = str(w_gdb / Path(file).stem) + "_AOIclip"
    #print(data_out)
    
    if arcpy.Exists(data_out):
        print("file already exists - skipped in terms of clipping")
        #repair geometery of file
        #arcpy.management.RepairGeometry(data_out, "DELETE_NULL", "ESRI")
        arcpy.management.RepairGeometry(data_out, "DELETE_NULL", "OGC")
        
        print("repair geometery run")
    else:
        print("clipping")
        arcpy.analysis.Clip(data_in, clip_area, data_out, None)
        #repair geometery of file
        #arcpy.management.RepairGeometry(data_out, "DELETE_NULL", "ESRI")
        arcpy.management.RepairGeometry(data_out, "DELETE_NULL", "OGC")
        
        print("repair geometery run")

In [ ]:
arcpy.env.workspace = str(w_gdb)

for zone in utm_zones_list:
    print(zone)
    
    #define inputs
    area_polys_elimf = str(w_gdb / ("area_polys_elimf_UTM" + str(zone))) #area polygons (output from eliminate)
    
    #file name UTM zone number is preceded by a 6 if N and 7 is S, determine this ready for list expression
    s_n = zone[-1]
    if s_n is 'N':
        zone_n = '6'    
    elif s_n is 'S':
        zone_n = '7'
    
    #find all clipped bf files for the country for the specified UTM zone
    expression = "*building*" + str(zone_n) + str(zone[:2]) + "*AOIclip"
    
    multifile_list = []
    for file in arcpy.ListFeatureClasses(expression): #find all files matching expression (UTM zone) 
        multifile_list.append(file) #and append to list
        
    print(multifile_list)
    
    if len(multifile_list) > 1: #if list has more than one item, i.e. there is more than one file for the utm zone, then merge
        print("there are multiple bf files per UTM zones - files need merging")
    else:
        print("only one bf file per UTM zones - no prior merging needed")
        bf_UTMzone = str(w_gdb / file)
        print(bf_UTMzone)
    
    ####
    #for the subset of polygons in each utm zone, check which utm zone polygons they overlap with (select by location)
    ####
    area_polys_subset = str(w_gdb / (str(iso) + "_area_polys_span_utm_zone_" + str(zone)))
    
    bf_extra_list = []
    
    if arcpy.Exists(area_polys_subset):
        print("area_polys_subset exists")
    
        #from the subset of area polygons, check which utm zone polygons they overlap with (select by location)
        utm_zones_shp = str(utmzones_folder / "utm_zones_lyr.shp")
        area_polys_subset_lyr = arcpy.MakeFeatureLayer_management(area_polys_subset, "area_polys_subset_lyr")
        utm_zones_span = arcpy.management.SelectLayerByLocation(utm_zones_shp, "CROSSED_BY_THE_OUTLINE_OF", area_polys_subset_lyr, None, "NEW_SELECTION", "NOT_INVERT")

        #add the UTM zone ID for the selected zones to a list
        utm_check_list = []
        with arcpy.da.SearchCursor(utm_zones_span, 'utm_zone') as cursor:
            for row in cursor:
                utm_check_list.append(row[0])        
        print(utm_check_list)

        #if length of list is >1, select the building footprint polygons from any additional utm zones
        if len(utm_check_list) > 1:
            utm_neighbour_list = utm_check_list
            utm_neighbour_list.remove(zone) #remove current zone from list

            #for remaining utm zones, 
            for item in utm_neighbour_list:
                print('item: {}'.format(item))
                
                #file name UTM zone number is preceded by a 6 if N and 7 is S, determine this ready for list expression
                s_n = item[-1]
                if s_n is 'N':
                    zone_n = '6'    
                elif s_n is 'S':
                    zone_n = '7'
                
                #find all clipped bf files for the country for the specified UTM zone
                expression = "*building*" + str(zone_n) + str(item[:2]) + "*AOIclip"
                
                for file in arcpy.ListFeatureClasses(expression): #find all files matching expression (UTM zone) 
                    #building footprint file
                    bf_file = file
                    print(file)
                        
                    #select any building footprints that are within subsetted area polygons
                    bf_sel = arcpy.management.SelectLayerByLocation(bf_file, "HAVE_THEIR_CENTER_IN", area_polys_subset, None, "NEW_SELECTION", "NOT_INVERT")
                    
                    n_selected = int(arcpy.GetCount_management(bf_sel)[0])
                    print('n_selected = {}'.format(n_selected))
                
                    #write out selected building footprints (those within one UTM zone but associated with area polygons from neighbour)
                    bf_subset = str(w_gdb / (str(iso) + "_bf_polys_from_utm_zone_" + str(item) + "_to_add_to_UTM_zone_" + str(zone)))
                    arcpy.CopyFeatures_management(bf_sel, bf_subset, '', None, None, None)
                    
                    bf_extra_list.append(bf_subset)
    
    ################
    #add in any building footprints from "neighbours" (due to different adm0 boundaries)
    #find the relevant (pre-processed) files and append to bf_extra_list
    for file_x in os.listdir(iso_bf_folder):
        
        #file name UTM zone number is preceded by a 6 if N and 7 is S, determine this ready for list expression
        s_n = zone[-1]
        if s_n is 'N':
            zone_n = '6'    
        elif s_n is 'S':
            zone_n = '7'
        
        expression = str(iso.upper()) + "*" + str(zone_n) + str(zone[:2]) + "_*shp"
        if fnmatch.fnmatch(file_x, expression):
            file_x_fc = str(w_gdb / (str(file_x[:-4])))
            arcpy.CopyFeatures_management(str(iso_bf_folder / file_x), file_x_fc, '', None, None, None)
            
            bf_extra_list.append(file_x_fc)
            print("BFs from neighbouring country dataset added: {}".format(file_x_fc))      

    ################
    
    #print(bf_extra_list)
    if len(bf_extra_list) > 0: #merge all additional building footprints
        print('length of bf_extra_list > 0: {}'.format(bf_extra_list))
                
        #file name UTM zone number is preceded by a 6 if N and 7 is S, determine this ready for list expression
        s_n = zone[-1] 
        if s_n is 'N':
            zone_n = '6'    
        elif s_n is 'S':
            zone_n = '7'
                
        #find all clipped bf files for the country for the specified UTM zone
        expression = "*building*" + str(zone_n) + str(zone[:2]) + "*AOIclip"
                
        for file in arcpy.ListFeatureClasses(expression): #find all files matching expression (UTM zone) 
            #building footprint file
            bf_file = file
        
            bf_extra_list.append(bf_file)
            arcpy.management.SelectLayerByAttribute(bf_file, "CLEAR_SELECTION", '', None) #clear previous selection
        
        #n_selected = int(arcpy.GetCount_management(bf_file)[0])
        #print('n_selected = {}'.format(n_selected))
        
        bf_UTMzone = str(bf_file) + "_w_extras"
        z = arcpy.management.Merge(bf_extra_list, bf_UTMzone)
        
        #n_selected = int(arcpy.GetCount_management(z)[0])
        #print('n_selected = {}'.format(n_selected))
    
    print( )
    
    intersect_list = [area_polys_elimf, bf_UTMzone] #list of features to intersect
    
    intersect_output = str(w_gdb / ("bf_" + str(zone) + "_area_polys_INTERSECT"))
    
    print(intersect_output)
    
    arcpy.analysis.Intersect(intersect_list, intersect_output, "ALL", None, "INPUT") #run intersect tool
    
    
    # Set output coordinate system
    utm_id = 'WGS 1984 UTM Zone ' + str(zone)
    #print(utm_id)
    outCS = arcpy.SpatialReference(utm_id)
    
    #calculate building footprint area
    arcpy.management.CalculateGeometryAttributes(intersect_output, "POLY_AREA AREA", '', "SQUARE_METERS", outCS, "SAME_AS_INPUT")                           
    
    print( )

Recalculate the area of building footprints to account for those that have been split

In [ ]:
for zone in utm_zones_list:
    
    # Set output coordinate system
    utm_id = 'WGS 1984 UTM Zone ' + str(zone)
    print(utm_id)
    outCS = arcpy.SpatialReference(utm_id)
    
    #specify dataset to calculate polygon area for and run tool
    intersect_output = str(w_gdb / ("bf_" + str(zone) + "_area_polys_INTERSECT"))
    print(intersect_output)
    arcpy.management.CalculateGeometryAttributes(intersect_output, "POLY_AREA AREA", '', "SQUARE_METERS", outCS, "SAME_AS_INPUT")

### 4. For each spatial unit, calculate the population density and proportion of area built

Summarise the total building area within each polygon (spatial join).

In [ ]:
for zone in utm_zones_list:
    
    area_polys_elimf = str(w_gdb / ("area_polys_elimf_UTM" + str(zone))) #area polygons (output from eliminate)
    intersect_output = str(w_gdb / ("bf_" + str(zone) + "_area_polys_INTERSECT")) #intersected building footprints
    
    targetFeatures = area_polys_elimf
    joinFeatures = intersect_output 
    out_feature_class = str(w_gdb / ("area_polys_bfareaSUM_UTM" + str(zone)))
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
    
    # Create new fieldmaps (1 for each output field required)
    fldmap_POLY_AREA_1 = arcpy.FieldMap()
    
    fldmap_POLY_AREA_1.addInputField(joinFeatures, "POLY_AREA") #does this need to be POLY_AREA1?
    
    # Set the merge rule to find the summed value of all bf in each feature
    fldmap_POLY_AREA_1.mergeRule = 'SUM'
    
    #set name
    f_name = fldmap_POLY_AREA_1.outputField
    f_name.name = 'SUM_POLY_AREA'
    f_name.aliasName = 'SUM_POLY_AREA'
    fldmap_POLY_AREA_1.outputField = f_name
    
    #Add all FieldMaps to the FieldMappings Object
    fieldmappings.addFieldMap(fldmap_POLY_AREA_1)
    
    #spatial join with summing of bf poly area    
    arcpy.analysis.SpatialJoin(targetFeatures, joinFeatures, out_feature_class, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings, "CONTAINS", None, '')
    print(out_feature_class)
    
    #change null values in SUM_POLY_AREA field to 0s
    field = 'SUM_POLY_AREA'
    with arcpy.da.UpdateCursor(out_feature_class, field) as cursor:
        for row in cursor:
            if row[0] == None:
                row[0] = 0
                
            cursor.updateRow(row) 

Calculate the necessary fields (proportion built, proportion not built etc)

In [ ]:
for zone in utm_zones_list:
    area_units = str(w_gdb / ("area_polys_bfareaSUM_UTM" + str(zone)))
    
    #calc proportion of polygon occupied by built structures
    arcpy.management.CalculateField(area_units, "BUILT_PROP", "!SUM_POLY_AREA!/!POLY_AREA!", "PYTHON3", '', "FLOAT")
    
    #calc proportion of polygon not occupied by built structures
    arcpy.management.CalculateField(area_units, "NBUILTPROP", "1-(!SUM_POLY_AREA!/!POLY_AREA!)", "PYTHON3", '', "FLOAT")

Change projection to WGS84

In [ ]:
for zone in utm_zones_list:
    
    area_units = str(w_gdb / ("area_polys_bfareaSUM_UTM" + str(zone)))
    area_units_WGS84 = str(area_units) + "_WGS84"
    
    # Set output coordinate system to WGS84
    outCS = arcpy.SpatialReference(4326)
    
    # run project tool
    arcpy.Project_management(area_units, area_units_WGS84, outCS)
    print(area_units_WGS84)

#NOTE: alias for new file doesn't update to file name when project is run

Merge spatial units from each UTM zone

In [ ]:
merge_list = []

for zone in utm_zones_list:
    area_units_WGS84 = str(w_gdb / ("area_polys_bfareaSUM_UTM" + str(zone))) + "_WGS84"
    merge_list.append(area_units_WGS84)

#merge polygon datasets in list
merged_output = str(w_gdb / (str(iso) + "_" + str((Path(area_units_WGS84).stem)[:-12]) + "WGS84"))
arcpy.management.Merge(merge_list, merged_output)   

Create pop density raster from pop count and raster of grid cell area

In [ ]:
pop_dens_null_raster = str(w_gdb / (str(iso) + "_pop_density_null")) 

outDivide = arcpy.sa.Divide(pop_raster, px_area_raster) #divide pop count by grid cell area
outDivide.save(pop_dens_null_raster)

#px_area units is m2. 

Convert null grid cell values to 0s

In [ ]:
pop_dens_zero_raster = str(w_gdb / (str(iso) + "_pop_density_zero")) 

output_raster = arcpy.sa.Con(arcpy.sa.IsNull(pop_dens_null_raster), 0, pop_dens_null_raster)
output_raster.save(pop_dens_zero_raster)

Convert pop density raster from pop per sqm to per sqkm

In [ ]:
pop_dens_sqkm_zero_raster = str(w_gdb / (str(iso) + "_pop_density_sqkm_zero")) 

popsqkm_raster = Raster(pop_dens_zero_raster) * 1000000
popsqkm_raster.save(pop_dens_sqkm_zero_raster)

Calculate average pop density within each spatial unit

In [ ]:
#create UID field
arcpy.management.CalculateField(merged_output, "UID", "!OBJECTID!", "PYTHON3", '', "TEXT")

#zonal statistics as table - MEAN
zs_table = str(w_gdb / (str(iso) + "_zs_mean_pop_density"))
arcpy.sa.ZonalStatisticsAsTable(merged_output, "UID", pop_dens_sqkm_zero_raster, zs_table, "DATA", "MEAN", "CURRENT_SLICE")

Check for spatial units with missing values

In [ ]:
#get count of number of polys for which zonal statistics calculated
zs_count = int(arcpy.GetCount_management(zs_table)[0]) 
total_poly_count = int(arcpy.GetCount_management(merged_output)[0]) 
print("In total, there are {} unit polygons, of which {} were recognised as zones in calculating mean pop density".format(zs_count, total_poly_count))
print("Therefore there are {} unit polygons for which mean pop density has not been calculated".format(str(total_poly_count-zs_count)))

#add new field for mean pop density
mean_pop_dens_field = "POP_DENS"
arcpy.AddField_management(merged_output, mean_pop_dens_field, "DOUBLE")

#join based on UID
in_field = "UID"
join_field = "UID"
zs_join = arcpy.AddJoin_management(merged_output, in_field, zs_table, join_field)

#update mean pop density field with calculated values
calc_expression = "!" + str(Path(zs_table).stem) + ".MEAN!"
arcpy.management.CalculateField(zs_join, mean_pop_dens_field, calc_expression, "PYTHON3", '', "FLOAT")

#remove join and write out to file
arcpy.RemoveJoin_management(zs_join)

zs_join_out = str(merged_output + '_zsjoin')
arcpy.CopyFeatures_management(zs_join, zs_join_out)

#select records where UID is null, i.e. no values calculated
expression = str(mean_pop_dens_field) + " IS NULL"
selected = arcpy.management.SelectLayerByAttribute(zs_join_out, "NEW_SELECTION", expression, None) 
zs_missing_count = int(arcpy.GetCount_management(selected)[0])
print("After joining, there are {} unit polygons for which mean pop density has not been calculated".format(str(total_poly_count-zs_count)))

Calculate mean pop density values for all spatial units with missing values

In [ ]:
#create centroids (point features) for each polygon
out_pts = str(w_gdb / "zs_missing_pnts")
arcpy.FeatureToPoint_management(selected, out_pts, "INSIDE")

#extract values to points
pts_values = str(w_gdb / "zs_missing_pnts_values")
arcpy.sa.ExtractValuesToPoints(out_pts, pop_dens_sqkm_zero_raster, pts_values, "INTERPOLATE")

Join values (calculated for centroid points) to spatial units and write out to file

In [ ]:
#join point values based on UID
in_field = "UID"
join_field = "UID"
extvals_join = arcpy.AddJoin_management(zs_join_out, in_field, pts_values, join_field)

#update mean pop density field with calculated values, first selecting just the null value to update
expression = str(Path(zs_join_out).stem) + "." + str(mean_pop_dens_field) + " IS NULL"
selected = arcpy.management.SelectLayerByAttribute(extvals_join, "NEW_SELECTION", expression, None) 
count = int(arcpy.GetCount_management(selected)[0])
print(count)

calc_expression = "!" + str(Path(pts_values).stem) + ".RASTERVALU!"
arcpy.management.CalculateField(extvals_join, mean_pop_dens_field, calc_expression, "PYTHON3", '', "FLOAT")

#clear selection, remove join and write out to file
arcpy.SelectLayerByAttribute_management(extvals_join, "CLEAR_SELECTION")
arcpy.RemoveJoin_management(extvals_join)

polys_calc_out = str(merged_output + '_popdens')
arcpy.CopyFeatures_management(extvals_join, polys_calc_out)

### 5. Classify the population density and built values, and calculate index

Add classification field - classify population density

In [ ]:
#classify mean pop density values

in_table = polys_calc_out
field = "BUILTscore"
expression = "reclass_nbuilt(!NBUILTPROP!)"
expression_type = "PYTHON3"
field_type = "SHORT"
codeblock = """
def reclass_nbuilt(nbuilt):
    if nbuilt == 1.0:
        return 0
    elif nbuilt >= 0.9 and nbuilt <1.0:
        return 1
    elif nbuilt >= 0.8 and nbuilt < 0.9:
        return 2
    elif nbuilt >= 0.7 and nbuilt < 0.8:
        return 3
    elif nbuilt >= 0.6 and nbuilt < 0.7:
        return 4
    elif nbuilt >= 0.5 and nbuilt < 0.6:
        return 5
    elif nbuilt >= 0.4 and nbuilt < 0.5:
        return 6
    elif nbuilt >= 0.3 and nbuilt < 0.4:
        return 7
    elif nbuilt >= 0.2 and nbuilt < 0.3:
        return 8
    elif nbuilt >= 0.1 and nbuilt < 0.2:
        return 9
    elif nbuilt < 0.1:
        return 10"""

arcpy.CalculateField_management(in_table, field, expression, expression_type, codeblock, field_type)

Add classification field - classify built proportion

In [ ]:
#classify non-built area values

in_table = polys_calc_out
field = "POPscore"
expression = "reclass_popdens(!POP_DENS!)"
expression_type = "PYTHON3"
field_type = "SHORT"
codeblock = """
def reclass_popdens(pop_dens):
    if pop_dens > 32075.01:
        return 10
    elif pop_dens > 18042.20 and pop_dens <= 32075.01:
        return 9
    elif pop_dens > 11547.01 and pop_dens <= 18042.20:
        return 8
    elif pop_dens > 8018.75 and pop_dens <= 11547.01:
        return 7
    elif pop_dens > 5891.33 and pop_dens <= 8018.75:
        return 6
    elif pop_dens > 4510.55 and pop_dens <= 5891.33:
        return 5
    elif pop_dens > 3563.89 and pop_dens <= 4510.55:
        return 4
    elif pop_dens > 2886.75 and pop_dens <= 3563.89:
        return 3
    elif pop_dens > 2385.74 and pop_dens <= 2886.75:
        return 2
    elif pop_dens > 0.0 and pop_dens <= 2385.74:
        return 1
    elif pop_dens == 0.0:
        return 0"""

arcpy.CalculateField_management(in_table, field, expression, expression_type, codeblock, field_type)

Calculate index values as the mean of pop and built scores

In [ ]:
in_table = polys_calc_out
field = "INDEXvalue"
expression = "0.5*(!POPscore! + !BUILTscore!)"
field_type = "DOUBLE"

arcpy.CalculateField_management(in_table, field, expression, None, None, field_type)

For all areas where grid cells with pop are missing from the constrained raster - set pop density and index score to be NoData

In [ ]:
#select all spatial units which contain building footprints from neighbouring country dataset (and are therefore missing pop estimates)
expression = str(iso.upper()) + "_to_add_to_*copy"
file_list = []

for file in arcpy.ListFeatureClasses(expression): 
    file_list.append(file)

if len(file_list) > 0:
    select_a = arcpy.management.SelectLayerByLocation(polys_calc_out, "CONTAINS", file, None, "NEW_SELECTION", "NOT_INVERT")
    for file in file_list:
        select_b = arcpy.management.SelectLayerByLocation(select_a, "CONTAINS", file, None, "ADD_TO_SELECTION", "NOT_INVERT")
    
    #set pop density score to NoData value
    in_table = select_b
    field = "POPscore"
    expression = "-99"

    arcpy.CalculateField_management(in_table, field, expression, None, None)


    #set index score to NoData value
    in_table = select_b
    field = "INDEXvalue"
    expression = "-99"

    arcpy.CalculateField_management(in_table, field, expression, None, None)


if len(file_list) > 0:
    #clear existing selection
    out = arcpy.SelectLayerByAttribute_management(select_b, "CLEAR_SELECTION")
else:
    out = arcpy.SelectLayerByAttribute_management(polys_calc_out, "CLEAR_SELECTION")

### 6. Post-processing - formatting of outputs

Clean up field names

In [ ]:
#update field name for polygon area (UNIT_AREA)
field = "POLY_AREA"
new_field_name = "UNIT_AREA"
arcpy.management.AlterField(out, field, new_field_name)

#update field name for sum_poly_area (BUILT_AREA)
field = "SUM_POLY_AREA"
new_field_name = "BUILT_AREA"
arcpy.management.AlterField(out, field, new_field_name)

In [ ]:
field_list_del = []

#list all fields. Except for specified fields, add field name to list of fields to be deleted
fields = arcpy.ListFields(out)
for field in fields:
    if field.name == "OBJECTID":
        print("{} is not deletable". format(field.name))
    elif field.name == "Shape":
        print("{} is not deletable". format(field.name))
    elif field.name == "Shape_Area":
        print("{} is not deletable". format(field.name))
    elif field.name == "Shape_Length":
        print("{} is not deletable". format(field.name))
    elif field.name == "uext_ID":
        print("{} - keep". format(field.name))
    elif field.name == "adm0_ISO3":
        print("{} - keep". format(field.name))
    elif field.name == "BUILT_PROP":
        print("{} - keep". format(field.name))
    elif field.name == "NBUILTPROP":
        print("{} - keep". format(field.name))
    elif field.name == "POP_DENS":
        print("{} - keep". format(field.name))
    elif field.name == "BUILTscore":
        print("{} - keep". format(field.name))
    elif field.name == "POPscore":
        print("{} - keep". format(field.name))
    elif field.name == "INDEXvalue":
        print("{} - keep". format(field.name))
    elif field.name == "UNIT_AREA":
        print("{} - keep". format(field.name))
    elif field.name == "BUILT_AREA":
        print("{} - keep". format(field.name))
    else:
        field_list_del.append(field.name) #append field to list

print("There are {} fields to be deleted".format(len(field_list_del)))
if len(field_list_del) > 0:
    arcpy.DeleteField_management(out, field_list_del) #delete fields
    
fields = arcpy.ListFields(out)
print("Final fields:")
for field in fields:
    print("{0} is a type of {1} with a length of {2}".format(field.name, field.type, field.length))

Remove polygons where total number of spatial units within uext (for the country of interest) is less than or equal to 30

In [ ]:
#create list of uext_ID values
uextid_list = []

with arcpy.da.SearchCursor(uext_poly, 'uext_ID') as cursor:
    for row in cursor:
        uextid_list.append(row[0])
        
uextid_del_list = []
#for each uext_ID value in the list, calculate the number of spatial units associated with that urban extent
for value in uextid_list:
    expression = "uext_ID = " + str(value)
    selected = arcpy.management.SelectLayerByAttribute(out, "NEW_SELECTION", expression, None) 
    count = int(arcpy.GetCount_management(selected)[0])
    #print(count)
    if count <= 30:
        uextid_del_list.append(value)

print(uextid_del_list)
#Delete the polygons associated with specified uextIDs in uextid_del_list
if len(uextid_del_list) > 0:
    for value in uextid_del_list:
        expression = "uext_ID = " + str(value)
        out_a = arcpy.management.SelectLayerByAttribute(out, "NEW_SELECTION", expression, None) 
        arcpy.DeleteFeatures_management(out_a)
else:
    out_a = out

In [ ]:
arcpy.SelectLayerByAttribute_management(out_a, "CLEAR_SELECTION")

Write out ease of social distancing index output to file

In [ ]:
#geodatabase version
file_out_fc = str(w_gdb / (str(iso.upper()) + "_ease_of_social_distancing_index_" + str(vers) + "_polygons"))
arcpy.CopyFeatures_management(out_a, file_out_fc)

#shapefile
file_out_shp = str(output_folder / (str(iso.upper()) + "_SocialDistancing_" + str(vers) + "_index.shp"))
arcpy.CopyFeatures_management(out_a, file_out_shp) 

field_list_del_shp = ["Shape_Area", "Shape_Leng"] #fields to delete
arcpy.DeleteField_management(file_out_shp, field_list_del_shp)

Create output point and polygon extent files

In [ ]:
#urban area polys
in_features = file_out_shp
out_file = str(output_folder / (str(iso.upper()) + "_SocialDistancing_" + str(vers) + "_urban_extents.shp"))
dissolve_field = "uext_ID"

arcpy.management.Dissolve(in_features, out_file, dissolve_field)

field = "adm0_ISO3"
expression = "str(iso)"
field_type = "TEXT"

arcpy.CalculateField_management(out_file, field, expression, None, None, field_type)

In [ ]:
#urban area names - select those within output urban area polys
uext_pnts = str(uext_iso_folder / ("ucdb_pts_SSA51_" + str(iso) + ".shp"))
select_pnts = arcpy.management.SelectLayerByLocation(uext_pnts, "WITHIN", out_file, None, "NEW_SELECTION", "NOT_INVERT")

#write selected points to temp file
file = str(w_gdb / "urb_pnts_tmp")
arcpy.CopyFeatures_management(select_pnts, file)

#update field name lat (GCPNT_LAT)
field = "GCPNT_LAT"
new_field_name = "PNT_LAT"
a = arcpy.management.AlterField(file, field, new_field_name, new_field_name)

#update field name lon (GCPNT_LON)
field = "GCPNT_LON"
new_field_name = "PNT_LON"
b = arcpy.management.AlterField(a, field, new_field_name, new_field_name)

#update field name country name (CTR_MN_NM)
field = "CTR_MN_NM"
new_field_name = "adm0_NAME"
c = arcpy.management.AlterField(b, field, new_field_name, new_field_name)

#update field name urban centre name (UC_NM_MN)
field = "UC_NM_MN"
new_field_name = "urb_NAME"
d = arcpy.management.AlterField(c, field, new_field_name, new_field_name)

field = "adm0_ISO3"
expression = "str(iso)"
field_type = "TEXT"

e = arcpy.CalculateField_management(d, field, expression, None, None, field_type)
  
#write out to file
out_pnts_shp = str(output_folder / (str(iso.upper()) + "_SocialDistancing_" + str(vers) + "_urban_points.shp"))
arcpy.CopyFeatures_management(e, out_pnts_shp)

Once this notebook has been run to completion, there will be 3 output files:
- XXX_SocialDistancing_v1_0_index.shp - the full set of spatial units (polygons) with ease of social distancing index values calculated. The INDEXvalue field provides the index values (0-10), calculated as the mean of the BUILTscore and POPscore field values. A value of 1 is indicative of relative ease of social distancing due to low population density and ample space around buildings. A value of 10 is indicative of high difficulty in maintaining social distancing due to very high population density and very little space around buildings. A no data value (-99) indicates missing data.
- XXX_SocialDistancing_v1_0_urban_extents.shp - polygons of the urban extents within which ease of social distancing index values were calculated
- XXX_SocialDistancing_v1_0_urban_points.shp - points representing urban centres, within each urban extent. Urban centre names and locations are derived from the GHS Urban Centre Database 2015 v1.2 (Florczyk et al., 2019a)